* Samuel L. Peoples

# Introduction

* In this project, Jeff Gordon's data is used from "A Core Gut Microbiome in Obese and Lean Twins" to determine if there exists a dissimilarity when comparing the Bray-Curtis-Faith beta diveristy between Lean-vs-Lean, Obese-vs-Obese, and Lean-vs-Obese samples.

# Import the appropriate packages

In [ ]:
from biom import load_table
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import random
import os

# Define the Single Rarefaction function

* This function is used to ensure the .biom file is properly sampled.

In [ ]:
def single_rarefaction(filepath):
    """
    Runs single_rarefaction.py and returns output file location
    Sample size set to 1000
    """
    input = '/study_77_011618-113533/processed_data/219_otu_table.biom'
    output = '/study_77_011618-113533/processed_data/219_otu_table_even1000.biom'
    sample_size = '1000'
    os.system('single_rarefaction.py'+
              ' -i '+filepath+input+
              ' -o '+filepath+output+
              ' -d '+sample_size)
    return output

# Definte the Table Parsing function

* This table is defined for any references that may be necessary

In [ ]:
def parse_table(filepath,rarefaction):
    """
    Passes the rarified table to be parsed
    Returns parsed table
    """
    # Load the table
    biom = load_table(filepath+rarefaction)
    # Define the observations and samples
    otus = biom.ids('observation')
    samples = biom.ids('sample')
    m = biom.matrix_data
    data = [pd.SparseSeries(m[i].toarray().ravel()) 
            for i in np.arange(m.shape[0])]
    table = pd.SparseDataFrame(data, index=otus, columns = samples)
    counts(table)
    return table

# Define the Counts function

* This function is helpful to verify that the sum of each column is the same

In [ ]:
def counts(table):
    """
    Counts the sum of the columns to verify 
    that the rarefaction had been completed properly.
    """
    for sum in table.sum(axis=0):
        if sum != 1000:
            print("single_rarefaction.py failure.")
    print("Rarefaction check completed.")

# Create and load the dissimilarity matrix using the betaDiversity function

* This function creates and returns a dissimilarity matrix based on the beta_diversity.py script

In [ ]:
def betaDiversity(filepath):
    """
    Runs beta_diversity.py with a bray_curtis_faith metric,
    and returns the beta diversity matrix as a DataFrame
    """
    input = '/study_77_011618-113533/processed_data/219_otu_table_even1000.biom'
    output_dir = '/study_77_011618-113533/processed_data/beta'
    output = '/bray_curtis_faith_219_otu_table_even1000.txt'
    metric = 'bray_curtis_faith'
    os.system('beta_diversity.py'
              +' -i '+filepath+input
              +' -o '+filepath+output_dir
              +' -m '+metric)
    return pd.DataFrame.from_csv(filepath+output_dir+output, sep ='\t')

# Populate the Lean and Obese labels for comparisons

* This function references the mapping file to create lists of Obese and Lean sample IDs.

In [ ]:
def label_tables(filepath, betas):
    """
    Returns label lists of sample IDs which 
    correspond to their values in the mapping file
    """
    ez_path = '/study_77_011618-113533/mapping_files/easy_mapping.csv'
    lean=[]
    obese=[]
    lines = []
    with open(filepath+ez_path,'r') as mapping:
        for line in mapping:
            lines.append(line.split(","))
        mapping.close()
    for line in lines:
        if str(line[2]).__contains__("Lean"):
            if line[0] in betas.keys():
                lean.append(line[0])
        elif str(line[2]).__contains__("Obese"):
            if line[0] in betas.keys():
                obese.append(line[0])
    return lean,obese

# Create the list of distances for Lean-vs-Lean, Obese-vs-Obese, and Lean-vs-Obese

* This function compares IDs and their categorical value of lean/ obese and appends them to the appropriate list of distances. These distances will be used to create boxplots for analysis.

In [ ]:
def break_table(betas, lean, obese):
    """
    Breaks the beta diversity matrix into Lean-vs-Lean, 
    Obese-vs-Obese, and Lean-vs-Obese. The function will
    not allow comparison of self (no distances of zero), 
    and will return lists of distances for each value.
    Each list is saved to its own csv file.
    """
    LL_stats = []
    OO_stats = []
    LO_stats = []

    for i in range(len(lean)):
        for j in range(len(betas[lean[i]].keys())):
            if betas[lean[i]].keys()[j] in lean:
                if betas[lean[i]][j] != 0:
                    LL_stats.append(betas[lean[i]][j])
            elif betas[lean[i]].keys()[j] in obese:
                LO_stats.append(betas[lean[i]][j])
    for i in range(len(obese)):
        for j in range(len(betas[obese[i]].keys())):
            if betas[obese[i]].keys()[j] in obese:
                if betas[obese[i]][j]!= 0:
                    OO_stats.append(betas[obese[i]][j])
    print("Lean-vs-Lean:\n "+
          "Low: "+str(min(LL_stats))+
          "\n High: "+str(max(LL_stats))+
          "\n Count: "+str(len(LL_stats)))
    print()
    print("Obese-vs-Obese:\n "+
          "Low: "+str(min(OO_stats))+
          "\n High: "+str(max(OO_stats))+
          "\n Count: "+str(len(OO_stats)))
    print()
    print("Lean-vs-Obese:\n "+
          "Low: "+str(min(LO_stats))+
          "\n high: "+str(max(LO_stats))+
          "\n Count: "+str(len(LO_stats)))
    with open("LL_stats.csv", "wb") as f:
        wr = csv.writer(f,quoting=csv.QUOTE_ALL)
        wr.writerow(LL_stats)
        f.close()
    with open("OO_stats.csv", "wb") as f:
        wr = csv.writer(f,quoting=csv.QUOTE_ALL)
        wr.writerow(OO_stats)
        f.close()
    with open("LO_stats.csv", "wb") as f:
        wr = csv.writer(f,quoting=csv.QUOTE_ALL)
        wr.writerow(LO_stats)
        f.close()
    return LL_stats, OO_stats, LO_stats

# Draw the boxplots

* Take a random sample of 1000 comparisons from each category and plot their boxplots

In [ ]:
def boxplots(LL_stats, OO_stats, LO_stats):
    """
    Defines data based on a random sampling of 1000 samples
    from each comparison, then saves the boxplots as "boxplot.png"
    """
    mpl.use('agg')
    data = sample(LL_stats, OO_stats, LO_stats)
    fig = plt.figure(1, figsize=(9,6))
    ax = fig.add_subplot(111)
    bp = ax.boxplot(data)
    ax.set_title('Comparison of Beta Diversity Dissimilarity\n Sample of 1000')
    ax.set_xticklabels(['Lean vs Lean','Obese vs Obese', 'Lean vs Obese'])
    [flier.set(marker='o',color='#e7298a', alpha=.3) for flier in bp['fliers']]
    fig.savefig("boxplot.png", bbox_inches='tight')

# Define the random sampling function

In [ ]:
def sample(LL_stats, OO_stats, LO_stats):
    """
    Returns an array with a 1000-random sample from each list
    """
    return [random.sample(LL_stats, 1000),
            random.sample(OO_stats, 1000),
            random.sample(LO_stats, 1000)]

# Define the main

In [ ]:
def main():
    """
    Define the path to the script, which should be in
    the same directory as study_77_011618-113533
    """
    filepath = '/home/qiime/Desktop/Project1'
    # Rarefaction of biom table, assigns path as variable
    rarefaction = single_rarefaction(filepath)
    # Parse tables, lists
    table = parse_table(filepath,rarefaction)
    betas = betaDiversity(filepath)
    lean, obese = label_tables(filepath, betas)
    LL_stats, OO_stats, LO_stats = break_table(betas, lean, obese)
    # Create the boxplots
    boxplots(LL_stats,OO_stats,LO_stats)
if __name__ == '__main__':
    main()

![](boxplot.png)

# Conclusion

* The outcome shows that when comparing Lean vs Lean and Obese vs Obese, the boxplots are quite similar, where the distribution is significantly different when comparing Lean vs Obese samples. This outcome will allow us to further investigate these dissimilarities in project 2, as well as determine the statistical significance of said dissimilarites.

* There is also room for improvement, where the use of a Uni-Frac beta diversity metric would introduce phylogeny, and provide a more accurate representation of the dissimilarity between samples. Improvements could also be made by running this same evaluation over a number of iterations, and averaging the results, as this is just one sample of 1000 comparisons, and a Monte-Carlo (or similar) implementation would improve the accuracy of the results.

* The ability to visually recognize the differences between Lean vs Obese and the other two cases suggests that there does exist a difference in the beta diversity between lean and obese patients, which implies that the microbial communities are equally different. 

* More values closer to zero suggests that the comparisons are more similar, where less Lean vs Obese having no values below .6 implies that the samples are more dissimilar. The statistical significance of this difference will be investigated further in Project 2.

# References

The following resources were used for the completion of this project

Qiime: [http://qiime.org/](http://qiime.org/)

Qiita: [https://qiita.ucsd.edu/](https://qiita.ucsd.edu/)

Gut Microbiome Dataset: [https://qiita.ucsd.edu/study/description/77](https://qiita.ucsd.edu/study/description/77)

Biom-Format: [http://biom-format.org/documentation/biom_format.html](http://biom-format.org/documentation/biom_format.html)

Lozupone/ Knight’s UniFrac: [http://aem.asm.org/content/71/12/8228.full](http://aem.asm.org/content/71/12/8228.full)

Qualitative/ Quantitative: [https://www.ncbi.nlm.nih.gov/pubmed/17220268](https://www.ncbi.nlm.nih.gov/pubmed/17220268)

Weighted UniFrac: [https://liorpachter.wordpress.com/2013/09/18/unifrac-revealed/](https://liorpachter.wordpress.com/2013/09/18/unifrac-revealed/)